In [1]:
# Imports for Tensor
import csv
import itertools
import math
import numpy as np
import os
import pandas as pd
import shutil
import sys
from collections import OrderedDict
from datetime import datetime
from tempfile import TemporaryDirectory
from typing import Tuple

from tqdm.auto import tqdm

import torch
import torch.nn.functional as F
from torch import nn, Tensor
from torch.utils.tensorboard import SummaryWriter
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from torchvision import transforms

from diffusers import StableDiffusionPipeline
from datasets import load_dataset

sys.path.append("../")

%load_ext autoreload
%autoreload 2

In [ ]:
# MetaDataset configuration
train_dataset_names = ('parkinsons', 'seed')
target_tasks = ("gender",)

val_set = MetaDataset(
    settings,
    train_dataset_names,
    split="val",
    tasks=target_tasks,
    target_format="tuple",
)

real_train_set = MetaDataset(
    settings,
    train_dataset_names,
    split="train",
    tasks=target_tasks,
    transform=randtxfm,
    target_format="tuple",
)

train_samp = MetaSampler(real_train_set, num_samples=len(real_train_set))


In [3]:
random_seed = 205 #205 Gave a good split for training
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Data Setup

In [4]:
# Datapaths
datadirs = {}
# datahome = '/data/shared/signal-diffusion'
datahome = '/mnt/d/data/signal-diffusion'

# Math dataset
datadirs['math'] = f'{datahome}/eeg_math'
datadirs['math-stft'] = os.path.join(datadirs['math'], 'stfts')

# Parkinsons dataset
datadirs['parkinsons'] = f'{datahome}/parkinsons/'
datadirs['parkinsons-stft'] = os.path.join(datadirs['parkinsons'], 'stfts')

#SEED dataset
datadirs['seed'] = f'{datahome}/seed/'
datadirs['seed-stft'] = os.path.join(datadirs['seed'], "stfts")

In [5]:
nsamps = 2000

preprocessor = GeneralPreprocessor(datadirs, nsamps, ovr_perc=0.5, fs=125) 
#preprocessor.preprocess(resolution=256, train_frac=0.8, val_frac=0.2, test_frac=0.0)

# Dataloader Setup

In [6]:
# Parameters
BATCH_SIZE = 64
SHUFFLE = True
NUM_WORKERS = 4
N_TOKENS = 128
RESOLUTION = 256
HOP_LENGTH = 80
persistent = NUM_WORKERS > 0

# Data augmentation
randtxfm = transforms.Compose([
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

In [7]:
# Datasets, excluding math
math_val_dataset = MathDataset(datadirs['math-stft'], split="val")
parkinsons_val_dataset = ParkinsonsDataset(datadirs['parkinsons-stft'], split="val")
seed_val_dataset = SEEDDataset(datadirs['seed-stft'], split="val")
val_datasets = [parkinsons_val_dataset, seed_val_dataset]

math_real_train_dataset = MathDataset(datadirs['math-stft'], split="train")
parkinsons_real_train_dataset = ParkinsonsDataset(datadirs['parkinsons-stft'], split="train", transform=None)
seed_real_train_dataset = SEEDDataset(datadirs['seed-stft'], split="train")
real_train_datasets = [parkinsons_real_train_dataset, seed_real_train_dataset]


val_set = GeneralDataset(val_datasets, split='val')
real_train_set = GeneralDataset(real_train_datasets, split='train')

# Sampler for balanced training data
train_samp = GeneralSampler(real_train_datasets, BATCH_SIZE, split='train')

In [8]:
# Dataloaders
val_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE,
                                         num_workers=NUM_WORKERS, pin_memory=True, 
                                         persistent_workers=persistent)
real_train_loader = torch.utils.data.DataLoader(real_train_set, batch_size=BATCH_SIZE, 
                                                num_workers=NUM_WORKERS, pin_memory=True, 
                                                persistent_workers=persistent, sampler=train_samp)

# Train

In [9]:
# define hyperparameters
OUTPUT_DIM = {"gender": 2, "health": 2, "emotion": 5}
DROPOUT = 0.7
BATCH_FIRST = True # True: (batch, seq, feature). False: (seq, batch, feature)

# CUDA for PyTorch
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
torch.backends.cudnn.benchmark = device.type == "cuda"

# Loss function
criterion = LabelSmoothingCrossEntropy(epsilon=0.2)

# Schedulers
scheduler = True
sched_gamma = 0.1

## Emotion

In [10]:
seed_val_dataset = SEEDDataset(datadirs['seed-stft'], split="val", task="emotion")
seed_real_train_dataset = SEEDDataset(datadirs['seed-stft'], split="train", task="emotion")
# Sampler for balanced training data
emotion_samp = EmotionSampler(datadirs['seed-stft'], BATCH_SIZE, split='train')

In [11]:
# Dataloaders
seed_val_loader = torch.utils.data.DataLoader(seed_val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE,
                                         num_workers=NUM_WORKERS, pin_memory=True, 
                                         persistent_workers=persistent)
seed_real_train_loader = torch.utils.data.DataLoader(seed_real_train_dataset, batch_size=BATCH_SIZE, 
                                                num_workers=NUM_WORKERS, pin_memory=True, 
                                                persistent_workers=persistent, sampler=emotion_samp)

In [12]:
model = ResNet(out_dim=OUTPUT_DIM['emotion'], layers=[1, 1, 2, 1])
model.to("cuda");

# SWA model instance
swa_model = torch.optim.swa_utils.AveragedModel(model)

In [13]:
# Runtime training parameters
opt, decay, restart, max_eta, decouple = (torch.optim.AdamW, 0.25, 0, None, True)

optimizer = opt(model.parameters(), lr=5e-5, weight_decay=decay)

# Learning rate scheduler
if scheduler == True:
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, sched_gamma, last_epoch=- 1, verbose=False)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=8)
    swa_scheduler = scheduler

    # = torch.optim.swa_utils.SWALR(optimizer, swa_lr=0.05)


# Create training configuration
swa_start=1000
ARGS = TrainingConfig(epochs=15, val_every_epochs=1, opt_restart_every=restart, swa_start=swa_start,
                      task=None)

# Log statistics
postfix = "_emotion"
comment = f"cnnclass_{model.name}_{str(type(optimizer)).split('.')[-1][:-2]}_decay{decay}{postfix}"
tbsw = SummaryWriter(log_dir="./tensorboard_logs/cnn/" + comment + "-" + 
                     datetime.now().isoformat(sep='_'), 
                     comment=comment)
print("#" * 80)
print("Training", comment)

# Training loop
losses, accs, val_accs = train_class(
    ARGS, model, swa_model,
    seed_real_train_loader, seed_val_loader,
    optimizer, scheduler, 
    criterion, device, tbsw
)

# load best model and evaluate on test set
model.load_state_dict(torch.load(f'best_model.pt'))
# test_loss, test_acc = evaluate_class(model, test_loader, criterion, device, 
#                                      tbsw, ARGS.epochs * len(real_train_loader) + 1)
# print(f'Test loss={test_loss:.3f}; test accuracy={test_acc:.3f}')

# Copy model to unique filename
os.makedirs("models", exist_ok=True)
shutil.copyfile("best_model.pt", f"models/best_model-{comment}.pt")
shutil.copyfile("last_model.pt", f"models/last_model-{comment}.pt")
print(f"Copied best model to models/best_model-{comment}.pt")

rmodel = model

In [14]:
cf, p = class_confusion(rmodel, seed_val_loader, emotion_class_labels, device, task="emotion")
plt.title("Emotion Validation")

## Gender

In [10]:
# Create model instance
model = CNNClassifierLight(1, OUTPUT_DIM, dropout=DROPOUT,pooling="max")
model = model.to(device)    

# SWA model instance
swa_model = torch.optim.swa_utils.AveragedModel(model)
swa_start = 12

# Runtime training parameters
opt, decay, restart, max_eta, decouple = (torch.optim.AdamW, 0.05, 0, None, True)

optimizer = opt(model.parameters(), lr=5e-4, weight_decay=decay)

# Learning rate scheduler
if scheduler == True:
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, sched_gamma, last_epoch=- 1, verbose=False)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=8)
    swa_scheduler = scheduler

    # = torch.optim.swa_utils.SWALR(optimizer, swa_lr=0.05)


# Create training configuration
ARGS = TrainingConfig(epochs=15, val_every_epochs=1, opt_restart_every=restart, swa_start=swa_start,
                      task="gender")

# Log statistics
postfix = ""
comment = f"cnnclass_{model.name}_{str(type(optimizer)).split('.')[-1][:-2]}_decay{decay}{postfix}"
tbsw = SummaryWriter(log_dir="./tensorboard_logs/cnn/" + comment + "-" + 
                     datetime.now().isoformat(sep='_'), 
                     comment=comment)
print("#" * 80)
print("Training", comment)

# Training loop
losses, accs, val_accs = train_class(
    ARGS, model, swa_model,
    real_train_loader, val_loader,
    optimizer, scheduler, 
    criterion, device, tbsw
)

# load best model and evaluate on test set
model.load_state_dict(torch.load(f'best_model.pt'))
# test_loss, test_acc = evaluate_class(model, test_loader, criterion, device, 
#                                      tbsw, ARGS.epochs * len(real_train_loader) + 1)
# print(f'Test loss={test_loss:.3f}; test accuracy={test_acc:.3f}')

# Copy model to unique filename
os.makedirs("models", exist_ok=True)
shutil.copyfile("best_model.pt", f"models/best_model-{comment}.pt")
shutil.copyfile("last_model.pt", f"models/last_model-{comment}.pt")
print(f"Copied best model to models/best_model-{comment}.pt")

rmodel = model

In [11]:
cf, p = class_confusion(rmodel, val_loader, general_class_labels, device)
plt.title("MAIN Validation")

In [12]:
fig = roc(model, val_loader, label="gender", task="gender")

# Train - Other Classes

In [13]:
# Disable grads for conv layers
for p in model.convs.parameters():
    p.requires_grad = False

## Health

In [14]:
model.to("cuda");

In [15]:
parkinsons_val_dataset = ParkinsonsDataset(datadirs['parkinsons-stft'], split="val", task="health")
parkinsons_real_train_dataset = ParkinsonsDataset(datadirs['parkinsons-stft'], split="train", transform=None, task="health")
# Sampler for balanced training data
health_samp = HealthSampler(datadirs['parkinsons-stft'], BATCH_SIZE, split='train')

In [16]:
# Dataloaders
parkinsons_val_loader = torch.utils.data.DataLoader(parkinsons_val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE,
                                         num_workers=NUM_WORKERS, pin_memory=True, 
                                         persistent_workers=persistent)
parkinsons_real_train_loader = torch.utils.data.DataLoader(parkinsons_real_train_dataset, batch_size=BATCH_SIZE, 
                                                num_workers=NUM_WORKERS, pin_memory=True, 
                                                persistent_workers=persistent, sampler=health_samp)

In [17]:
# Runtime training parameters
opt, decay, restart, max_eta, decouple = (torch.optim.AdamW, 0.05, 0, None, True)

optimizer = opt(model.fcs["health"].parameters(), lr=5e-4, weight_decay=decay)

# Learning rate scheduler
if scheduler == True:
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, sched_gamma, last_epoch=- 1, verbose=False)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=8)
    swa_scheduler = scheduler

    # = torch.optim.swa_utils.SWALR(optimizer, swa_lr=0.05)


# Create training configuration
swa_start=1000
ARGS = TrainingConfig(epochs=75, val_every_epochs=1, opt_restart_every=restart, swa_start=swa_start,
                      task="health")

# Log statistics
postfix = "_health"
comment = f"cnnclass_{model.name}_{str(type(optimizer)).split('.')[-1][:-2]}_decay{decay}{postfix}"
tbsw = SummaryWriter(log_dir="./tensorboard_logs/cnn/" + comment + "-" + 
                     datetime.now().isoformat(sep='_'), 
                     comment=comment)
print("#" * 80)
print("Training", comment)

# Training loop
losses, accs, val_accs = train_class(
    ARGS, model, swa_model,
    parkinsons_real_train_loader, parkinsons_val_loader,
    optimizer, scheduler, 
    criterion, device, tbsw
)

# load best model and evaluate on test set
model.load_state_dict(torch.load(f'best_model.pt'))
# test_loss, test_acc = evaluate_class(model, test_loader, criterion, device, 
#                                      tbsw, ARGS.epochs * len(real_train_loader) + 1)
# print(f'Test loss={test_loss:.3f}; test accuracy={test_acc:.3f}')

# Copy model to unique filename
os.makedirs("models", exist_ok=True)
shutil.copyfile("best_model.pt", f"models/best_model-{comment}.pt")
shutil.copyfile("last_model.pt", f"models/last_model-{comment}.pt")
print(f"Copied best model to models/best_model-{comment}.pt")

rmodel = model

In [18]:
cf, p = class_confusion(rmodel, parkinsons_val_loader, health_class_labels, device, task="health")
plt.title("Health Validation")

In [19]:
fig = roc(model, parkinsons_val_loader, label="health", task="health")
